In [2]:
import pandas as pd

In [3]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

In [4]:
def pil_to_cv(pil_image):
    open_cv_image = np.array(pil_image)
    return cv.cvtColor(open_cv_image, cv.COLOR_RGB2BGR)

In [5]:
from ultralyticsplus import YOLO, render_result

model = YOLO('foduucom/plant-leaf-detection-and-classification')
model.overrides['conf'] = 0.25  # NMS confidence threshold
model.overrides['iou'] = 0.45  # NMS IoU threshold
model.overrides['agnostic_nms'] = False  # NMS class-agnostic
model.overrides['max_det'] = 1000  # maximum number of detections per image

c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\USER\AppData\Local\Programs\Python\Python311\Lib\site-packages\ultralytics\nn\tasks.py:340: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user v

In [12]:
from PIL import Image

img = cv.imread('../asset/comparator/Ripening.jpeg.jpg')

results = model.predict(img)

# observe results
print(results[0].boxes)
render = render_result(model=model, image=img, result=results[0])
render.show()


0: 640x480 4 apples, 216.5ms
Speed: 7.0ms preprocess, 216.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


tensor([[3.40000e+02, 0.00000e+00, 4.80000e+02, 1.08000e+02, 6.51079e-01, 2.80000e+01],
        [4.16000e+02, 3.99000e+02, 4.80000e+02, 5.60000e+02, 4.56879e-01, 2.80000e+01],
        [2.37000e+02, 0.00000e+00, 3.06000e+02, 1.90000e+01, 4.24905e-01, 2.80000e+01],
        [3.10000e+02, 0.00000e+00, 4.63000e+02, 4.20000e+01, 2.66538e-01, 2.80000e+01]])


In [7]:
for result in results:
    boxes = result.boxes  # Access bounding boxes
    for box in boxes:
        class_id = int(box.cls)  # Class ID
        label = model.names[class_id]  # Map class ID to label
        print(label) 

mango
